In [1]:
import pandas as pd
import re
from tqdm import tqdm
from ast import literal_eval
from urllib.parse import urlparse

In [2]:
df = pd.read_csv('./data_all/urls-with-redirection.csv')

In [3]:
df

,original_url,redirect_urls,final_url,source
0,https://alii.pub/6o3dam,['https://alii.pub/6o3dam'\n 'https://allshops...,https://www.aliexpress.com/item/10050041493828...,YouTube Search
1,https://www.pinterest.com/singlemommywinning/,NaN,https://www.pinterest.com/singlemommywinning/,YouTube Search
2,http://rstyle.me/n/b9i9d29vde,['http://rstyle.me/n/b9i9d29vde'],https://rstyle.me/n/b9i9d29vde,YouTube Search
3,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,NaN,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,YouTube Search
4,https://bit.ly/valencia_tuscany,['https://bit.ly/valencia_tuscany'],https://us.valenciatheaterseating.com/collecti...,YouTube Search
...,...,...,...,...
753233,https://t.co/YJRLLtBAuR,"['https://t.co/YJRLLtBAuR', 'https://shp.ee/44...",https://idgame.shopee.co.id/getvouchers/6725dd...,Twitter Search
753234,https://t.co/9ogADdxF7O,"['https://t.co/9ogADdxF7O', 'https://shp.ee/tp...",https://shopee.co.id/product/20480005/23671843...,Twitter Search
753235,https://t.co/Sum8Z2CGl6,"['https://t.co/Sum8Z2CGl6', 'https://shp.ee/44...",https://shopee.co.id/product/421797229/2304634...,Twitter Search
753236,https://t.co/5cd28dNtPK,"['https://t.co/5cd28dNtPK', 'https://bit.ly/3l...",https://virustopcaviteph.blogspot.com/2022/02/...,Twitter Search


In [4]:
df['is_affiliate'] = 0

In [5]:
df

,original_url,redirect_urls,final_url,source,is_affiliate
0,https://alii.pub/6o3dam,['https://alii.pub/6o3dam'\n 'https://allshops...,https://www.aliexpress.com/item/10050041493828...,YouTube Search,0
1,https://www.pinterest.com/singlemommywinning/,NaN,https://www.pinterest.com/singlemommywinning/,YouTube Search,0
2,http://rstyle.me/n/b9i9d29vde,['http://rstyle.me/n/b9i9d29vde'],https://rstyle.me/n/b9i9d29vde,YouTube Search,0
3,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,NaN,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,YouTube Search,0
4,https://bit.ly/valencia_tuscany,['https://bit.ly/valencia_tuscany'],https://us.valenciatheaterseating.com/collecti...,YouTube Search,0
...,...,...,...,...,...
753233,https://t.co/YJRLLtBAuR,"['https://t.co/YJRLLtBAuR', 'https://shp.ee/44...",https://idgame.shopee.co.id/getvouchers/6725dd...,Twitter Search,0
753234,https://t.co/9ogADdxF7O,"['https://t.co/9ogADdxF7O', 'https://shp.ee/tp...",https://shopee.co.id/product/20480005/23671843...,Twitter Search,0
753235,https://t.co/Sum8Z2CGl6,"['https://t.co/Sum8Z2CGl6', 'https://shp.ee/44...",https://shopee.co.id/product/421797229/2304634...,Twitter Search,0
753236,https://t.co/5cd28dNtPK,"['https://t.co/5cd28dNtPK', 'https://bit.ly/3l...",https://virustopcaviteph.blogspot.com/2022/02/...,Twitter Search,0


In [6]:
with open('./substr集合.txt', 'r', encoding='utf-8') as f:
    substrings=f.readlines()
substrings = [s.strip() for s in substrings]
substring_pattern = '|'.join(substrings)

In [7]:
def contain_substring(url):
    result = re.search(substring_pattern, url)
    if result:
        return True
    return False

In [8]:
new_keywords = []

In [9]:
keywords = ['tracking_id', 'clickid3', 'click_id3',
    'clickid2', 'click_id2', 'click_id', 'clickid', 'irclickid2', 
    'irclick_id', 'irclickid', 'affiliate_transaction', 
    'affiliate_campaign', 'sub_affiliate', 
    'affiliate', 'aff_code', 'subaffiliate', 'affiliate_code', 
    'coupon_code', 'affiliate_url', 'affiliate_ref', 'affiliate_source', 
    'affiliate_partner', 'affiliate_banner', 'affiliate_pub', 'aff', 'affid',
    'affiliateid', 'affiliate_id', 'affiliate_link', 'affiliate_track', 'aff_id',
    'affiliate_channel', 'af'
]

In [10]:
def parameter_check(url):
    parsed_url = urlparse(url)
    # 获取URL的参数信息
    query = parsed_url.query
    params_kv = re.split(r"[\?&;]+", query)
    try:
        params_kv.remove('')
    except:
        pass
    # print(params_kv)
    for kv in params_kv:
        try:
            k = kv.split('=')[0]
            v = kv.split('=')[1]
            if k.lower() in keywords:
                # print(k)
                return True
            for part in k.split('_'):
                for w in re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', part):
                    if w.lower() == 'aff' or w.lower() == 'affiliate':
                        new_keywords.append(k)
                        # print(k)
                        return True
            for part in v.split('_'):
                for w in re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?=[A-Z]|$)', part):
                    if w.lower() == 'aff' or w.lower() == 'affiliate':
                        # print(kv)
                        return True
        except:
            pass

    return False

In [11]:
def aff_url_identify(url):
    if contain_substring(url):
        return True
    if parameter_check(url):
        return True
    return False

In [12]:
for i in tqdm(range(len(df))):
    urls = []
    urls.append(df.at[i, 'original_url'])
    urls.append(df.at[i, 'final_url'])
    try:
        urls += literal_eval(df.at[i,'redirect_urls'])
    except:
        pass
    for url in urls:
        if aff_url_identify(url):
            df.at[i, 'is_affiliate'] = 1
            break

  0%|          | 720/753238 [00:00<01:44, 7182.61it/s]

100%|██████████| 753238/753238 [01:22<00:00, 9150.70it/s] 


In [17]:
new_keywords = list(set(new_keywords))

In [18]:
new_keywords

['affiliate-code',
 'aff_offer_id',
 'fpm-affiliate-utm-source',
 'aff_ref',
 'isAffiliate',
 'affExt',
 'aw_affiliate',
 'sub_aff_id',
 'subAffId',
 'affiliate_banner_id',
 'epic_affiliate',
 'aff_trace_key',
 'affExtPa',
 'aff_network',
 'aff_request_id',
 'affiliateNetwork',
 'SubAffiliateID',
 'PAffiliateID',
 'aff_sub_id',
 'aff_name',
 'aff_klick_id',
 'affiliateRef',
 'aff_sub',
 'super_affiliate_code',
 'aff_pid',
 'aff_short_key',
 'aff%5C_id',
 'aff_expired_f',
 'affiliateN',
 'subAffiliateID',
 'aff_click_id',
 'aff_adid',
 'aff_platform',
 'AffiliateReferenceID',
 'aff_sub4',
 'AffiliateCode',
 'aff_source',
 'is_affiliate',
 'aff_key',
 'id_affiliate',
 'aff_fcid',
 'cf_affiliate_id',
 'affiliateCode',
 'FpAffiliate',
 '2p_affiliate_id']

In [19]:
df.to_csv('./data_all/urls-with-redirection-labeled.csv', index=False)

In [21]:
df[df['is_affiliate']==1]

,original_url,redirect_urls,final_url,source,is_affiliate
0,https://alii.pub/6o3dam,['https://alii.pub/6o3dam'\n 'https://allshops...,https://www.aliexpress.com/item/10050041493828...,YouTube Search,1
2,http://rstyle.me/n/b9i9d29vde,['http://rstyle.me/n/b9i9d29vde'],https://rstyle.me/n/b9i9d29vde,YouTube Search,1
3,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,NaN,https://rstyle.me/+IMNF676MkVHNh9aOf9TaEQ,YouTube Search,1
6,https://go.magik.ly/ml/e66t/,NaN,https://go.magik.ly/ml/e66t/,YouTube Search,1
8,https://NordVPN.com/madewithlau,['https://nordvpn.com/madewithlau'\n 'https://...,https://nordvpn.com/creator/entertainment/?cou...,YouTube Search,1
...,...,...,...,...,...
753230,https://t.co/GkEDvPr74X,"['https://t.co/GkEDvPr74X', 'https://shp.ee/9q...",https://shopee.co.id/product/271297865/1779790...,Twitter Search,1
753232,https://t.co/Yn0XMsPO4o,"['https://t.co/Yn0XMsPO4o', 'https://shopee.co...",https://sv.shopee.co.id/referral/homeTrending?...,Twitter Search,1
753233,https://t.co/YJRLLtBAuR,"['https://t.co/YJRLLtBAuR', 'https://shp.ee/44...",https://idgame.shopee.co.id/getvouchers/6725dd...,Twitter Search,1
753234,https://t.co/9ogADdxF7O,"['https://t.co/9ogADdxF7O', 'https://shp.ee/tp...",https://shopee.co.id/product/20480005/23671843...,Twitter Search,1


In [22]:
124462/753238

0.16523595463850735